In [ ]:
import dask.dataframe as dd
from dask import delayed
import dask
import math

!! testout everything here and let it run in the cloud !!

In [ ]:
data = dd.read_csv('first_10partitions.csv')
data = data.drop('Unnamed: 0', axis=1)

numSamples = data.shape[0].compute()
idx = int(numSamples*0.7)
train = data.loc[0:idx]
test = data.loc[idx:numSamples]

# browsing data

In [ ]:
batch = train.head(500)
batch.isna().sum().any()

In [ ]:
feat60 = ['state_t', 'state_q0001','state_q0002','state_q0003','state_u','state_v','pbuf_ozone','pbuf_CH4','pbuf_N2O']
feat1 = ['state_ps','pbuf_SOLIN','pbuf_LHFLX','pbuf_SHFLX','pbuf_TAUX','pbuf_TAUY','pbuf_COSZRS','cam_in_ALDIF','cam_in_ALDIR','cam_in_ASDIF','cam_in_ASDIR','cam_in_LWUP','cam_in_ICEFRAC','cam_in_LANDFRAC','cam_in_OCNFRAC','cam_in_SNOWHLAND']

target60 = ['ptend_t','ptend_q0001','ptend_q0002','ptend_q0003','ptend_u','ptend_v']
target1 = ['cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

features60 = [] 
for f in feat60:
    features60 = features60 + [f+'_'+str(i) for i in range(60)]
allF = features60 + feat1

targets60 = [] 
for f in target60:
    targets60 = targets60 + [f+'_'+str(i) for i in range(60)]
allT = targets60 + target1

len(allF) + len(allT), batch.shape

In [ ]:
targetsToDrop12 = [ 'ptend_q0001', 'ptend_q0002', 'ptend_q0003', 'ptend_u', 'ptend_v']
dropT = ['ptend_q0002_12','ptend_q0002_13','ptend_q0002_14','ptend_q0002_15'] # attention, I think i also need to predict _15
for f in targetsToDrop12:
    dropT = dropT + [f+'_'+str(i) for i in range(12)]

allT2 = [i for i in allT if i not in dropT]

# normalizing data

In [ ]:
mean_values = train[allF+allT].mean().compute()
std_values = train[(allF+allT)].std().compute()

In [ ]:
allFZ = []
for col in allF:
    newF = col+'_zScore'
    train[newF] = (train[col] - mean_values[col]) / std_values[col]
    test[newF] = (test[col] - mean_values[col]) / std_values[col]
    allFZ.append(newF)

allTZ = []
for col in allT:
    newF = col+'_zScore'
    train[newF] = (train[col] - mean_values[col]) / std_values[col]
    test[newF] = (test[col] - mean_values[col]) / std_values[col]
    allTZ.append(newF)

# do it batchwise

In [ ]:
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
def getXy(partId, train, feat, tf):
    X = train[feat].get_partition(partId).compute()
    y = train[tf].get_partition(partId).compute()
    # normalize
    for f in feat:
        X[f] = (X[f] - mean_values[f]) / std_values[f]
    for f in tf:
        y[f] = (y[f] - mean_values[f]) / std_values[f]

    return lgb.Dataset(X, label=y)

In [ ]:
lgb_model = lgb.LGBMRegressor(objective='regression', metric='l2', verbose=-1)
multi_output_model = MultiOutputRegressor(lgb_model)

train_data = getXy(0, train, allF, allT2)
initial_model = multi_output_model.fit(train_data.data, train_data.label)

for i in range(1,train.npartitions):
    train_data = getXy(i, train, allF, allT2)
    updated_model = multi_output_model.fit(lgb_model.get_params(), train_data, num_boost_round=100, init_model=initial_model.estimators_[0].booster_)
    initial_model.estimators_[0].booster_ = booster
# Get final model
final_model = initial_model
    


In [ ]:
for f in train_data.label.columns:
    if train_data.label[f].isna().any():
        print(f, mean_values[f], std_values[f])

In [ ]:
for target in target_cols:
    print(f"Training model for target: {target}")
    
    # Train initial model on first batch
    train_data = lgb.Dataset(batches[0][0], label=batches[0][1][target])
    params = {'objective': 'regression', 'metric': 'l2'}
    initial_model = lgb.train(params, train_data, num_boost_round=100)
    initial_model.save_model(f'model_{target}.txt')
    
    # Train on remaining batches
    for i in range(1, len(batches)):
        train_data = lgb.Dataset(batches[i][0], label=batches[i][1][target])
        updated_model = lgb.train(params, train_data, num_boost_round=100, 
                                  init_model=f'model_{target}.txt', 
                                  keep_training_booster=True)
        updated_model.save_model(f'model_{target}.txt')
    
    # Store the final model
    models[target] = updated_model